In [ ]:
import pandas as pd
import requests
import json
import sqlite3
import shutil

In [ ]:
# Функция для получения токена доступа
def get_access_token():
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    
    payload = {
        'scope': 'GIGACHAT_API_PERS'
    }
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': '24f98fbe-1b03-46ed-97bf-d52ae27ad01f',
        'Authorization': f'Basic {CREDENTIALS}'  # Замените CREDENTIALS на ваши данные
    }
    
    response = requests.post(url, headers=headers, data=payload, verify=False)
    return response.json()['access_token']

# Получаем токен доступа
ACCESS_TOKEN = get_access_token()

In [ ]:
# Функция для запроса GigaChat с конкретными атрибутами
def gigachat_generate_category(title: str, description: str):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    
    message = f"""На основе информации о товаре с названием '{title}' и описанием '{description}', 
    выдели до 10 атрибутов и их значения в формате JSON: 
    {{"параметры": {{"параметр1": "значение1", "параметр2": "значение2"}}}}.
    Примеры атрибутов: количество (шт.), цвет, материал, вес, размер, объем, мощность, длина, ширина, высота и так далее. 
    Выдели какие могут быть параметры у данного товара, а также их значения."""

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "user",
                "content": message
            }
        ],
        "stream": False,
        "repetition_penalty": 1
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-Client-ID': CLIENT_ID,  # Замените CLIENT_ID на ваш client ID
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

    response = requests.post(url, headers=headers, data=payload, verify=False)
    return json.loads(response.json()['choices'][0]['message']['content'])

In [ ]:
# Подключение к базе данных и загрузка данных
conn = sqlite3.connect('/repo/converted_database.db')
query = "SELECT ОКПД2, Параметры FROM MTR"
mtr_data = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
# Список для хранения результатов
results = []

# Проход по товарам
for index, row in mtr_data.iterrows():
    okpd2_code = row['ОКПД2']
    parameters = row['Параметры']  # Получаем параметры из колонки

    # Запрос к GigaChat для получения атрибутов
    try:
        attributes = gigachat_generate_category(okpd2_code, parameters)
        
        # Преобразуем атрибуты и их значения в JSON
        attributes_json = json.dumps(attributes['параметры'], ensure_ascii=False)
        
        # Добавляем JSON строку с новыми атрибутами
        row['generated_attributes'] = attributes_json
        results.append(row)
    except Exception as e:
        print(f"Ошибка при обработке ОКПД2 '{okpd2_code}': {e}")

# Преобразуем результаты в DataFrame
result_df = pd.DataFrame(results)

# Сохраняем результаты в новый CSV
output_file = '/repo/okpd2_with_generated_attributes.csv'
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Результаты сохранены в {output_file}")


In [ ]:
# Путь к базе данных
db_copy_path = '/repo/converted_database_copy.db'

# Чтение данных из CSV файлов
csv_with_attributes = '/repo/okpd2_with_generated_attributes.csv'
csv_best_matches = '/repo/okpd2_best_matches.csv'
attributes_df = pd.read_csv(csv_with_attributes)
matches_df = pd.read_csv(csv_best_matches)

In [ ]:
# Объединение DataFrame по ОКПД2
merged_df = pd.merge(attributes_df, matches_df, left_on='ОКПД2', right_on='OKPD2', how='left')

# Подготовка новых колонок
# Конкатенируем все уровни, чтобы сформировать полный путь категории
merged_df['новый признак'] = merged_df[['level_1_category', 'level_2_category', 
                                         'level_3_category', 'level_4_category', 
                                         'level_5_category', 'level_6_category']].apply(
    lambda x: ' > '.join(x.dropna().astype(str)), axis=1)

# Подготавливаем новую колонку JSON с категориями
merged_df['новые категории JSON'] = merged_df.apply(
    lambda x: json.dumps({
        "категории": {
            "level_1": x['level_1_category'],
            "level_2": x['level_2_category'],
            "level_3": x['level_3_category'],
            "level_4": x['level_4_category'],
            "level_5": x['level_5_category'],
            "level_6": x['level_6_category']
        }
    }, ensure_ascii=False), axis=1)

In [ ]:
# Подключение к базе данных и обновление таблицы MTR
with sqlite3.connect(db_copy_path) as conn:
    # Добавление новых колонок, если они не существуют
    try:
        conn.execute('ALTER TABLE MTR ADD COLUMN "новый признак" TEXT')
    except sqlite3.OperationalError:
        print("Столбец 'новый признак' уже существует.")
        
    try:
        conn.execute('ALTER TABLE MTR ADD COLUMN "новые категории JSON" TEXT')
    except sqlite3.OperationalError:
        print("Столбец 'новые категории JSON' уже существует.")
    
    # Обновление таблицы новыми значениями
    for index, row in merged_df.iterrows():
        conn.execute("""UPDATE MTR
                        SET "новый признак" = ?, "новые категории JSON" = ?
                        WHERE ОКПД2 = ?""",
                      (row['новый признак'], row['новые категории JSON'], row['ОКПД2']))
    conn.commit()

print(f"Новые колонки добавлены и обновлены в базе данных по пути {db_copy_path}.")
